In [1]:
!nvidia-smi

Sat Nov  9 13:16:33 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:31:00.0 Off |                    0 |
| N/A   41C    P0             27W /   72W |    3185MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           AuthenticAMD
CPU family:          25
Model:               1
Model name:          AMD EPYC 7R13 Processor
Stepping:            1
CPU MHz:             2819.871
BogoMIPS:            5300.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            512K
L3 cache:            16384K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl nonstop_tsc cpuid extd_apicid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy

In [153]:
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [4]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
# download the Jigsaw Toxic dataset from https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data

In [6]:
!pwd

/home/ec2-user/SageMaker/BERT


In [7]:
!ls

BOW_and_SVM.ipynb				   test.csv
fine_tuning_bert.ipynb				   test.csv.zip
fine-tunned-bert-model-dict.pth			   test_labels.csv
fine-tunned-bert-model.pth			   test_labels.csv.zip
jigsaw-toxic-comment-classification-challenge.zip  train.csv
sample_submission.csv				   train.csv.zip
sample_submission.csv.zip


In [7]:
!unzip -q -o jigsaw-toxic-comment-classification-challenge.zip
!unzip -q -o train.csv.zip
!unzip -q -o test.csv.zip
!unzip -q -o sample_submission.csv.zip
!unzip -q -o test_labels.csv.zip

In [8]:
df = pd.read_csv("./train.csv")
df[df['identity_hate'] == 1].head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
105,00472b8e2d38d1ea,A pair of jew-hating weiner nazi schmucks.,1,0,1,0,1,1
176,006b94add72ed61c,I think that your a Fagget get a oife and burn...,1,0,1,1,1,1


In [9]:
df[(df['identity_hate'] == 1) & (df['severe_toxic'] == 1)].head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
887,026bd33490542b2e,"you gay motherfucker i know where you live,i a...",1,1,1,0,1,1
1017,02c6e41e4b317ac3,WOULDN'T BE THE FIRST TIME BITCH. FUCK YOU I'L...,1,1,1,1,1,1
1116,03088aacc4600323,"Fuck you faggot, I don't care if i get blocked...",1,1,1,0,1,1
1312,039296aa294ee10b,"SHUT UP, YOU FAT POOP, OR I WILL KICK YOUR ASS!!!",1,1,1,1,1,1
2359,065675c12e22e3e1,You fascit Nazi! You socialist bastard! You fi...,1,1,1,0,1,1
2374,0661f992b32b3351,FUCK OF NAZI PROPAGANDA\nodpierdalcie sie jeba...,1,1,1,0,0,1
2937,07f1e0a1c10fcf28,Dumb Niggaz \n\nYall Niggas are dumb.Sean King...,1,1,1,0,1,1
3008,081a5f7b052138cf,Hey Roy. Go fukk yourself you gay bich.\n\nYa...,1,1,1,0,1,1
3469,0959d4fdaf74a050,THIS DUDE THAT BLOCKED ME IS A FUCKING COCK SU...,1,1,1,0,1,1
6063,10304d5db5fb5821,n i g g e r f a g g o t,1,1,0,0,0,1


In [10]:
new_df = df[(df['obscene'] == 0) & (df['severe_toxic'] == 0)].copy()
new_df = new_df[['comment_text', 'toxic']]
new_df

,comment_text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [11]:
count_vectorizer = CountVectorizer()
X_count_vectorizer = count_vectorizer.fit_transform(new_df['comment_text'])

In [12]:
count_vectorizer.get_feature_names_out()[10000:10100], len(count_vectorizer.vocabulary_.keys())

(array(['abate', 'abated', 'abaya', 'abb', 'abba', 'abbas',
        'abbas_and_the_armenians', 'abbasaheb', 'abbasgulu', 'abbasid',
        'abbasids', 'abbass', 'abbassed', 'abbassi', 'abbassid',
        'abbastanza', 'abbau', 'abbe', 'abberations', 'abberline',
        'abberrant', 'abbes', 'abbetting', 'abbey', 'abbeymaynard',
        'abbeys', 'abbi', 'abbie', 'abbitt', 'abbot', 'abbott',
        'abbottabad', 'abbottsford', 'abbr', 'abbr2', 'abbrasive',
        'abbrev', 'abbrevation', 'abbreveated', 'abbreviate',
        'abbreviated', 'abbreviatedis', 'abbreviates', 'abbreviating',
        'abbreviation', 'abbreviations', 'abbrevs', 'abbriviations',
        'abbrégé', 'abbs', 'abbusively', 'abby', 'abbyses', 'abbythecat',
        'abbywinters', 'abbás', 'abc', 'abc123xyz', 'abc2', 'abcd',
        'abcde', 'abce2', 'abcedare', 'abcedere', 'abcef', 'abclocal',
        'abcmonster', 'abcnews', 'abd', 'abd5f5', 'abdali', 'abdallah',
        'abdallar', 'abdaly', 'abdalyar', 'abdel',

In [13]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf_vectorize = tfidf_vectorizer.fit_transform(new_df['comment_text'])

In [14]:
tfidf_vectorizer.get_feature_names_out()[10000:10100], len(tfidf_vectorizer.vocabulary_.keys())

(array(['abate', 'abated', 'abaya', 'abb', 'abba', 'abbas',
        'abbas_and_the_armenians', 'abbasaheb', 'abbasgulu', 'abbasid',
        'abbasids', 'abbass', 'abbassed', 'abbassi', 'abbassid',
        'abbastanza', 'abbau', 'abbe', 'abberations', 'abberline',
        'abberrant', 'abbes', 'abbetting', 'abbey', 'abbeymaynard',
        'abbeys', 'abbi', 'abbie', 'abbitt', 'abbot', 'abbott',
        'abbottabad', 'abbottsford', 'abbr', 'abbr2', 'abbrasive',
        'abbrev', 'abbrevation', 'abbreveated', 'abbreviate',
        'abbreviated', 'abbreviatedis', 'abbreviates', 'abbreviating',
        'abbreviation', 'abbreviations', 'abbrevs', 'abbriviations',
        'abbrégé', 'abbs', 'abbusively', 'abby', 'abbyses', 'abbythecat',
        'abbywinters', 'abbás', 'abc', 'abc123xyz', 'abc2', 'abcd',
        'abcde', 'abce2', 'abcedare', 'abcedere', 'abcef', 'abclocal',
        'abcmonster', 'abcnews', 'abd', 'abd5f5', 'abdali', 'abdallah',
        'abdallar', 'abdaly', 'abdalyar', 'abdel',

In [214]:
smaller_df = new_df[:50000]
smaller_df[smaller_df['toxic'] == 1]

X_train, X_test, y_train, y_test = train_test_split(np.array(smaller_df['comment_text']), np.array(smaller_df['toxic']), test_size=0.3, random_state=42)
pd.DataFrame(y_train).value_counts()

0    33238
1     1762
dtype: int64

In [215]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf_vectorize = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf_vectorize = tfidf_vectorizer.transform(X_test)

In [216]:
# the distribution of the data is 1 toxic for each ~20 non toxic so we need to assign a higher weight to the toxic (class 1)
svm = SVC(class_weight={0: 1, 1: 20}) 
svm.fit(X_train_tfidf_vectorize, y_train)

SVC(class_weight={0: 1, 1: 20})

In [217]:
svm.predict(tfidf_vectorizer.transform(['pretty'])), svm.predict(tfidf_vectorizer.transform(['idiot']))

(array([0]), array([1]))

In [218]:
y_pred = svm.predict(X_test_tfidf_vectorize)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98     14246
           1       0.81      0.28      0.41       754

    accuracy                           0.96     15000
   macro avg       0.89      0.64      0.70     15000
weighted avg       0.96      0.96      0.95     15000



In [234]:
def is_toxic(model, vectorizer, text, print_score=False):
    encoded_input = vectorizer.transform([text])
    if print_score:
        return model.predict(encoded_input)[0] == True, model.predict(encoded_input), model.decision_function(encoded_input)

    return model.predict(encoded_input)[0] == True, model.predict(encoded_input)


def prefixed_is_toxic_check(model, vectorizer):
    def print_is_toxic(text):
        result = is_toxic(model, vectorizer, text, True)
        return f'[Score: {result[2][0]:0.3f}] [Toxic: {result[0]}] {text}'

    return print_is_toxic("You are an idiot"), \
        print_is_toxic("You are a devil"), \
        print_is_toxic("You stink"), \
        print_is_toxic("What are you doing here? You are not welcome in this country"), \
        print_is_toxic("hello my friend! I love you, you know that, right?"), \
        print_is_toxic("I watched the soccer game yesterday and it was an aweful experience"), \
        print_is_toxic("Hello Dad; how are you doing?"), \
        print_is_toxic("Hi Mom!")


In [235]:
prefixed_is_toxic_check(svm, tfidf_vectorizer)

('[Score: 3.441] [Toxic: True] You are an idiot',
 '[Score: -0.407] [Toxic: False] You are a devil',
 '[Score: -0.538] [Toxic: False] You stink',
 '[Score: -0.591] [Toxic: False] What are you doing here? You are not welcome in this country',
 '[Score: -0.737] [Toxic: False] hello my friend! I love you, you know that, right?',
 '[Score: -1.077] [Toxic: False] I watched the soccer game yesterday and it was an aweful experience',
 '[Score: -0.382] [Toxic: False] Hello Dad; how are you doing?',
 '[Score: 0.217] [Toxic: True] Hi Mom!')

In [212]:
smaller_df = new_df.copy() # LinearSVC is pretty fast so no need to make it smaller

X_train, X_test, y_train, y_test = train_test_split(np.array(smaller_df['comment_text']), np.array(smaller_df['toxic']), test_size=0.3, random_state=42)
pd.DataFrame(y_train).value_counts()

0    100581
1      5149
dtype: int64

In [213]:
linear_svm_tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf_vectorize = linear_svm_tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf_vectorize = linear_svm_tfidf_vectorizer.transform(X_test)

# the distribution of the data is 1 toxic for each ~20 non toxic so we need to assign a higher weight to the toxic (class 1)
linear_svm = LinearSVC(max_iter=5000, class_weight={0: 1, 1: 20})
linear_svm.fit(X_train_tfidf_vectorize, y_train)

y_pred = linear_svm.predict(X_test_tfidf_vectorize)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.97     43173
           1       0.49      0.63      0.55      2141

    accuracy                           0.95     45314
   macro avg       0.73      0.80      0.76     45314
weighted avg       0.96      0.95      0.95     45314



In [237]:
prefixed_is_toxic_check(linear_svm, linear_svm_tfidf_vectorizer)

('[Score: 5.884] [Toxic: True] You are an idiot',
 '[Score: 0.816] [Toxic: True] You are a devil',
 '[Score: 0.985] [Toxic: True] You stink',
 '[Score: -0.544] [Toxic: False] What are you doing here? You are not welcome in this country',
 '[Score: -1.257] [Toxic: False] hello my friend! I love you, you know that, right?',
 '[Score: -2.353] [Toxic: False] I watched the soccer game yesterday and it was an aweful experience',
 '[Score: -0.395] [Toxic: False] Hello Dad; how are you doing?',
 '[Score: 0.088] [Toxic: True] Hi Mom!')